In [ ]:
'''
import re 
pattern = re.compile(r'.*?(?:\. |\n|$)', re.DOTALL)

def extract_xml_cot(text: str) -> str:
    think = text.rsplit("<think>")[-1]
    think = think.rsplit("<answer>")[0]
    return think 

#TODO: must be checked again...!!
def make_latent_data(step, data):
    
    assert step > 0, "step should be greater than 0"
    
    temp_prompts, temp_labels, answer_labels = [], [], []
    
    for pred in data:
        temp_prompts.append(pred.rsplit("<think>", 1)[0] + "<think>")
        ## ... <think><-| ... </think>
        temp_labels.append(extract_xml_cot(pred))
        ## <think> |-> ... </think> ... <-|<answer>
        answer_labels.append("<answer>" + pred.rsplit("<answer>", 1)[-1])
        ## |-><answer> ... </answer>|
    
    final_inputs, final_labels, num_steps = [], [], []
    for t_p, t_l, o_l in zip(temp_prompts, temp_labels, answer_labels):
        
        temp_steps = [m.group() for m in pattern.finditer(t_l) if m.group()]
        
        ## Case 1) len(temp_steps) > step:
        ## e.g., temp_steps = ['0xxxx. ', '1xxxx\n', '2xxxx. ', '3xxxx\n'] & step = 3
        ## => <|start-latent|><|latent|><|latent|><|latent|><|end-latent|> + '3xxxx\n' (temp_steps[step:])
        ## final_inputs.append(t_p + "<|start-latent|>" + "<|latent|>" * step + "<|end-latent|>")
        ## final_labels.append(temp_steps[step] + t_l.split(temp_steps[step:])[-1] + o_l)
        
        ## Case 2) len(temp_steps) < step:
        ## e.g., temp_steps = ['0xxxx. ', '1xxxx\n'] & step = 3
        ## => <|start-latent|><|latent|><|latent|><|latent|> 
        ## step = len(temp_steps)
        ## final_inputs.append(t_p + "<|start-latent|>" + "<|latent|>" * step + "<|end-latent|>")
        ## final_labels.append(o_l)
        
        if step >= len(temp_steps):
            final_inputs.append(t_p + "<|start-latent|>" + "<|latent|>" * len(temp_steps) + "<|end-latent|>")
            final_labels.append(o_l)
        
        else:
            final_inputs.append(t_p + "<|start-latent|>" + "<|latent|>" * step + "<|end-latent|>" )
            final_labels.append(temp_steps[step] + t_l.split(temp_steps[step])[-1] + o_l)
        
        ## For checking,
        num_steps.append(len(temp_steps))
    
    return final_inputs, final_labels, num_steps
'''

## Make Pretraining Dataset

In [6]:
import pandas as pd 

base_train = pd.read_csv("./data/processed/base/answer_kl/train.csv")
base_valid = pd.read_csv("./data/processed/base/answer_kl/valid.csv")

In [7]:
base_train.head(2)

,question,true_answer,pred_answer,conf_labels,input_prompt,answer_label,confidence_input_prompt,confidence_label
0,<|begin_of_text|><|start_header_id|>system<|en...,"For every 5 kangaroos, there is one koala, mea...","{question}\n<think>First, we need to find out ...",60,<|begin_of_text|><|start_header_id|>system<|en...,<answer>There are 216 koalas and kangaroos alt...,<|begin_of_text|><|start_header_id|>system<|en...,<confidence>60</confidence>
1,<|begin_of_text|><|start_header_id|>system<|en...,Jennifer has 2*10 = <<2*10=20>>20 apples.\nShe...,"{question}\n<think>\nTo solve this problem, we...",90,<|begin_of_text|><|start_header_id|>system<|en...,"<answer>\nJennifer has 8 pears, 18 oranges, an...",<|begin_of_text|><|start_header_id|>system<|en...,<confidence>90</confidence>


In [8]:
base_train_preds = list(base_train['input_prompt'])
base_valid_preds = list(base_valid['input_prompt'])

base_train_labels = list(base_train['answer_label'])
base_valid_labels = list(base_valid['answer_label'])

In [9]:
LATENT_START = "<|start_latent|>"
LATENT_END   = "<|end_latent|>"
LATENT_TOK   = "<|latent|>"

ASSIST_TAG = "<|start_header_id|>assistant<|end_header_id|>"

def split_assistant(pred):
    if ASSIST_TAG not in pred:
        raise ValueError("assistant header not found")
    pre, post = pred.split(ASSIST_TAG, 1)
    return pre + ASSIST_TAG, post.lstrip("\n")

def mask_first_n_lines(text: str, n: int):
    lines = text.splitlines(keepends=True)
    n_lat = min(n, len(lines))
    stub  = LATENT_START + LATENT_TOK * n_lat + LATENT_END
    remain = "".join(lines[n_lat:])
    return stub #+ remain

def make_latent_data(step: int, samples):
    inputs, labels, counts = [], [], []
    for pred in samples:
        prefix, assistant_body = split_assistant(pred)
        counts.append(len(assistant_body.splitlines()))
        
        masked_assist = mask_first_n_lines(assistant_body, step)
        
        inputs.append(prefix + "\n" + masked_assist)
        
        label_part = "\n".join(assistant_body.splitlines()[step:])
        labels.append(label_part)
    return inputs, labels, counts

In [10]:
train_inputs, train_labels, train_steps = make_latent_data(6, base_train_preds)
valid_inputs, valid_labels, valid_steps = make_latent_data(6, base_valid_preds)

In [11]:
import numpy as np 
train_steps.index(int(np.max(train_steps)))

1482

In [12]:
print(base_train['input_prompt'][1482])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Apr 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a conversation between **User** and **Assistant**.
The User asks a question, and the Assistant provides a solution.
Before answering, the Assistant reasons through the problem step-by-step.
The reasoning is enclosed within `<think> ... </think>`, and the final answer within `<answer> ... </answer>`.

Example:
{question}
<think>{step-by-step reasoning}</think>
<answer>{final answer}</answer>

Now, respond to the following using the **exact same format**:
Mark is running for an election and wins 70% of the votes in an area with 100,000 voters.  He got twice as many total votes in the remaining area.  How many votes did he get in total?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{question}
<think>
To solve this problem, we need to break it down into smaller steps. Let

In [13]:
import numpy as np

mean = np.mean(train_steps)
median = np.median(train_steps)
std = np.std(train_steps)
q25 = np.quantile(train_steps, 0.25)
q75 = np.quantile(train_steps, 0.75)
max = np.max(train_steps)

print(f"Mean: {mean}")
print(f"Median: {median}")
#print(f"Std: {std}")
print(f"25% Quantile: {q25}")
print(f"75% Quantile: {q75}")
print(f"MAX: {max}")

print("="*30)

mean = np.mean(valid_steps)
median = np.median(valid_steps)
std = np.std(valid_steps)
q25 = np.quantile(valid_steps, 0.25)
q75 = np.quantile(valid_steps, 0.75)
max = np.max(valid_steps)

print(f"Mean: {mean}")
print(f"Median: {median}")
#print(f"Std: {std}")
print(f"25% Quantile: {q25}")
print(f"75% Quantile: {q75}")
print(f"MAX: {max}")

Mean: 8.893547427891763
Median: 8.0
25% Quantile: 5.0
75% Quantile: 11.0
MAX: 95
Mean: 8.682730923694779
Median: 8.0
25% Quantile: 5.0
75% Quantile: 11.0
MAX: 48


In [14]:
## 0 -> 1
## 1 -> 2
## 2 -> 3
## 4,5,6 -> 4
## 7,8,9 -> 5
## 10,11,12 -> 6
## 13,14,15 -> 7
## 16,17,18 -> 8
## 19,20,21 -> 9
## 22,23,24 -> 111

stages = [i+1 for i in range(9)] + [95]


In [15]:
for step in [i+1 for i in range(9)] + [95]:
    
    train_inputs, train_labels, train_steps = make_latent_data(step, base_train_preds)
    valid_inputs, valid_labels, valid_steps = make_latent_data(step, base_valid_preds)
    
    train_df = pd.DataFrame({
        'input_prompt': train_inputs, 
        'answer_label': [l1 + l2 for l1, l2 in zip(train_labels, base_train_labels)], 
    })
    
    valid_df = pd.DataFrame({
        'input_prompt': valid_inputs, 
        'answer_label': [l1 + l2 for l1, l2 in zip(valid_labels, base_valid_labels)],
    })
    
    train_df.to_csv(f"./data/prev/prev/coconut/stage_{step}_train.csv", index=False)
    valid_df.to_csv(f"./data/prev/prev/coconut/stage_{step}_valid.csv", index=False)


In [16]:
train_df.head(2)

,input_prompt,answer_label
0,<|begin_of_text|><|start_header_id|>system<|en...,<answer>There are 216 koalas and kangaroos alt...
1,<|begin_of_text|><|start_header_id|>system<|en...,"<answer>\nJennifer has 8 pears, 18 oranges, an..."


In [21]:
print(train_df['input_prompt'][2])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Apr 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a conversation between **User** and **Assistant**.
The User asks a question, and the Assistant provides a solution.
Before answering, the Assistant reasons through the problem step-by-step.
The reasoning is enclosed within `<think> ... </think>`, and the final answer within `<answer> ... </answer>`.

Example:
{question}
<think>{step-by-step reasoning}</think>
<answer>{final answer}</answer>

Now, respond to the following using the **exact same format**:
A busy restaurant is counting how many customers they had during that Friday to try to predict how many they might get on Saturday. During breakfast, they had 73 customers. During lunch, they had 127 customers. During dinner, they had 87 customers. If they predict they'll get twice the amount of customers on Saturday as they had

In [22]:
print(train_df['answer_label'][3])

<answer>
64
</answer>


## Evaluation

In [14]:
import pandas as pd 

valid_data = pd.read_csv("./data/prev/prev/coconut/stage_8_valid.csv")

In [2]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

from src.llm_model_utils import (
    create_model,
    create_tokenizer
    )   

model_name = "meta-llama/Llama-3.2-3B-Instruct"
tokenizer = create_tokenizer(model_name)
model = create_model(
    model_name,
    tokenizer=tokenizer,
    use_int8=False,
    device_map="auto",
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_tokens("<|start-latent|>")
tokenizer.add_tokens("<|end-latent|>")
tokenizer.add_tokens("<|latent|>")

latent_id = tokenizer.convert_tokens_to_ids("<|latent|>")
start_id = tokenizer.convert_tokens_to_ids("<|start-latent|>")
end_id = tokenizer.convert_tokens_to_ids("<|end-latent|>")

model.resize_token_embeddings(len(tokenizer))
embeddings = model.get_input_embeddings()
target_id = tokenizer.convert_tokens_to_ids("<<")

for token_id in [latent_id, start_id, end_id]:
    target_embedding = embeddings.weight.data[target_id] 
    embeddings.weight.data[token_id] = target_embedding
    lm_head = model.lm_head
    lm_head.weight.data[token_id] = lm_head.weight.data[target_id]

/mnt/home/chaeyun-jang/.conda/envs/llm_ft/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.68s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [3]:
from src.peft_utils import get_lora_model

query_peft_dir = "/mnt/home/chaeyun-jang/reasoning_calibration/logs/2025-05-10T18-27-05"

model = get_lora_model(
    model,
    peft_id_or_dir=query_peft_dir,
    is_trainable=False,
    adapter_name="query",
)

/mnt/home/chaeyun-jang/.conda/envs/llm_ft/lib/python3.12/site-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.query.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.query.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.query.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.query.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.query.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.query.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.query.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.query.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.query.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.query.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.query.weight', 'base_model.model.model.layers.2.self_attn

In [4]:
import torch
from src.coconut import Coconut

coconut_model = Coconut(model, latent_id, start_id, end_id, tokenizer.eos_token_id)

In [20]:
input_ids = tokenizer(valid_data['input_prompt'][1], return_tensors="pt").to('cuda')
input_ids['position_ids'] = torch.tensor(list(range(len(input_ids['input_ids'][0])))).unsqueeze(0).to('cuda')

model.config.pad_token_id = tokenizer.pad_token_id
output = coconut_model.generate(**input_ids, max_new_tokens=100)

In [21]:
print(valid_data['input_prompt'][1])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Apr 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a conversation between **User** and **Assistant**.
The User asks a question, and the Assistant provides a solution.
Before answering, the Assistant reasons through the problem step-by-step.
The reasoning is enclosed within `<think> ... </think>`, and the final answer within `<answer> ... </answer>`.

Example:
{question}
<think>{step-by-step reasoning}</think>
<answer>{final answer}</answer>

Now, respond to the following using the **exact same format**:
James creates a media empire.  He creates a movie for $2000.  Each DVD cost $6 to make.  He sells it for 2.5 times that much.  He sells 500 movies a day for 5 days a week.  How much profit does he make in 20 weeks?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
<|start_latent|><|latent|><|latent|><|latent|><|latent|><|la

In [22]:
print(valid_data['answer_label'][1])

Selling price per DVD = 2.5 * $6 = $15
Next, we need to find the total revenue per day. James sells 500 movies per day, so we multiply the selling price by the number of movies sold.
Total revenue per day = 500 * $15 = $7500
Since James sells 500 movies a day for 5 days a week, we multiply the total revenue per day by 5.
Total revenue per week = $7500 * 5 = $37,500
Now, we need to find the total revenue for 20 weeks. We multiply the total revenue per week by 20.
Total revenue for 20 weeks = $37,500 * 20 = $750,000
Next, we need to find the total cost for 20 weeks. James spends $6 per DVD, and he sells 500 DVDs per day for 5 days a week. So, the total cost per week is:
Total cost per week = 500 * $6 = $3000
Total cost for 20 weeks = $3000 * 20 = $60,000
Finally, we can find the profit by subtracting the total cost from the total revenue.
Profit = Total revenue - Total cost
Profit = $750,000 - $60,000
Profit = $690,000
</think><answer>
$690,000
</answer>


In [24]:
print(tokenizer.decode(output[0]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 17 Apr 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

This is a conversation between **User** and **Assistant**.
The User asks a question, and the Assistant provides a solution.
Before answering, the Assistant reasons through the problem step-by-step.
The reasoning is enclosed within `<think>... </think>`, and the final answer within `<answer>... </answer>`.

Example:
{question}
<think>{step-by-step reasoning}</think>
<answer>{final answer}</answer>

Now, respond to the following using the **exact same format**:
James creates a media empire.  He creates a movie for $2000.  Each DVD cost $6 to make.  He sells it for 2.5 times that much.  He sells 500 movies a day for 5 days a week.  How much profit does he make in 20 weeks?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
<|start_latent|><|latent|><|latent|><|latent|